In [1]:
import os
import json
from string import ascii_letters

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import umap
from sklearn.metrics import f1_score, accuracy_score
from sklearn.manifold import TSNE
import torch
from datasets import DatasetDict, Dataset, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
sns.set_theme()

## Чтение и подготовка данных

In [2]:
df = pd.read_csv('./data/ЕНСТРУ.csv')

df.drop(columns=["Отдел", "Группа", "Класс", "Вид", "Подвид"], inplace=True)
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)

In [3]:
values = df['Ведомственный классификатор'].value_counts()
labels = values[values >= 3].index.tolist()

df = df[df['Ведомственный классификатор'].isin(labels)]

In [4]:
# Создаем словари для соответствия метка -> id и обратно
labels = sorted(df['Ведомственный классификатор'].unique())
label2id = {label: int(idx) for idx, label in enumerate(labels)}
id2label = {int(idx): label for label, idx in label2id.items()}

In [5]:
# Сохранение в файл
with open('./data/id2label.json', 'w', encoding='utf-8') as f:
    json.dump(id2label, f)

In [6]:
df['text'] = df['Наименование с характеристикой']
df['label'] = df['Ведомственный классификатор'].apply(lambda x: label2id[x])
num_labels = len(set(labels))

In [7]:
# Преобразуем DataFrame в датасет Hugging Face
dataset = Dataset.from_pandas(df)

# Приводим столбец "label" к типу ClassLabel, чтобы можно было использовать stratify_by_column
class_label = ClassLabel(names=labels)
dataset = dataset.cast_column("label", class_label)

Casting the dataset:   0%|          | 0/32973 [00:00<?, ? examples/s]

In [8]:
temp_test_dataset = dataset.train_test_split(test_size=0.1, stratify_by_column="label", seed=42)
train_val_dataset = temp_test_dataset['train'].train_test_split(test_size=0.33333, stratify_by_column="label", seed=42)

dataset = DatasetDict({
    'train': train_val_dataset['train'],
    'val': train_val_dataset['test'],
    'test': temp_test_dataset['test']})

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Наименование', 'Код', 'Характеристика', 'Наименование с характеристикой', 'Ведомственный классификатор', 'text', 'label', '__index_level_0__'],
        num_rows: 19783
    })
    val: Dataset({
        features: ['Наименование', 'Код', 'Характеристика', 'Наименование с характеристикой', 'Ведомственный классификатор', 'text', 'label', '__index_level_0__'],
        num_rows: 9892
    })
    test: Dataset({
        features: ['Наименование', 'Код', 'Характеристика', 'Наименование с характеристикой', 'Ведомственный классификатор', 'text', 'label', '__index_level_0__'],
        num_rows: 3298
    })
})

In [10]:
dataset['train'].to_pandas().to_csv('./data/train.csv', index=False)
dataset['val'].to_pandas().to_csv('./data/val.csv', index=False)
dataset['test'].to_pandas().to_csv('./data/test.csv', index=False)

## Загрузка модели

In [11]:
# Функция токенизации (объединяем поля "question" и "answer")
def tokenize_function(batch):
    #print(batch)
    return tokenizer(batch["text"], truncation=True, max_length=512, padding="max_length")

In [12]:
model_name = "intfloat/multilingual-e5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

/home/schetinnikov.k/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at intfloat/multilingual-e5-base were not used when initializing XLMRobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weight

Map:   0%|          | 0/19783 [00:00<?, ? examples/s]

Map:   0%|          | 0/9892 [00:00<?, ? examples/s]

Map:   0%|          | 0/3298 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Наименование', 'Код', 'Характеристика', 'Наименование с характеристикой', 'Ведомственный классификатор', 'text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 19783
    })
    val: Dataset({
        features: ['Наименование', 'Код', 'Характеристика', 'Наименование с характеристикой', 'Ведомственный классификатор', 'text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 9892
    })
    test: Dataset({
        features: ['Наименование', 'Код', 'Характеристика', 'Наименование с характеристикой', 'Ведомственный классификатор', 'text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 3298
    })
})

## Обучение модели

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "f1": f1}

In [15]:
early_stopper = EarlyStoppingCallback(
    early_stopping_patience=5,
    # early_stopping_threshold=0.05 # you can change this value if needed
)

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,          # валидация каждые 1000 шагов
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    bf16=True,                # использование bf16, если поддерживается оборудованием
    logging_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

/home/schetinnikov.k/.local/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/schetinnikov.k/.local/lib/python3.10/site-packages/transformers/trainer.py:2700: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Step,Training Loss,Validation Loss,Accuracy,F1
500,6.353000,5.445633,0.237768,0.128477
1000,5.244800,4.660322,0.342903,0.237597
1500,4.534600,4.123012,0.404772,0.301272
2000,4.077200,3.728711,0.456834,0.359197
2500,3.665400,3.399779,0.484129,0.385881
3000,3.315300,3.135273,0.528407,0.443045
3500,3.030200,2.928184,0.547614,0.465335
4000,2.772600,2.741084,0.579660,0.502535
4500,2.560600,2.581213,0.592196,0.519474
5000,2.360900,2.443959,0.611302,0.542471


/home/schetinnikov.k/.local/lib/python3.10/site-packages/transformers/trainer.py:2700: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/home/schetinnikov.k/.local/lib/python3.10/site-packages/transformers/trainer.py:2700: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/home/schetinnikov.k/.local/lib/python3.10/site-packages/transformers/trainer.py:2700: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/home/schetinnikov.k/.local/lib/python3.10/site-packages/transformers/trainer.py:2700: FutureWarning: `torch.cuda.amp.autoca

TrainOutput(global_step=30950, training_loss=1.139501113521841, metrics={'train_runtime': 6815.7219, 'train_samples_per_second': 145.128, 'train_steps_per_second': 4.541, 'total_flos': 2.643011909571072e+17, 'train_loss': 1.139501113521841, 'epoch': 50.0})

## Проверка на тесте

In [19]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import torch
from tqdm import tqdm

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "intfloat/multilingual-e5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained('./results/checkpoint-30500', num_labels=num_labels).to(device)
model.eval();

In [21]:
# 2. Функция для предсказаний
def evaluate(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device)
            }
            outputs = model(**inputs)
            
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch["label"].cpu().numpy())
    
    return np.array(predictions), np.array(true_labels)

In [22]:
# 1. Подготовка DataLoader для теста
test_dataloader = torch.utils.data.DataLoader(
    tokenized_datasets["test"],
    batch_size=32,
    shuffle=False
)

# 3. Запуск оценки
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
predictions, true_labels = evaluate(model, test_dataloader, device)

Evaluating: 100%|██████████| 104/104 [00:21<00:00,  4.75it/s]


In [23]:
target_names = [id2label[id] for id in true_labels]

In [24]:
# 4. Детальные метрики

print("\nClassification Report:")
print(classification_report(
    true_labels, 
    predictions, 
))


Classification Report:
              precision    recall  f1-score   support

           6       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         1
          16       0.67      1.00      0.80         2
          17       0.50      1.00      0.67         1
          19       1.00      1.00      1.00         1
          22       1.00      1.00      1.00         1
          24       0.00      0.00      0.00         1
          25       0.50      1.00      0.67         2
          26       1.00      1.00      1.00         1
          27       0.00      0.00      0.00         0
          31       1.00      1.00      1.00         1
          32       1.00      1.00      1.00         1
          34       1.00      1.00      1.00         1
          43       0.00      0.00      0.00         0
   

/home/schetinnikov.k/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/schetinnikov.k/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/schetinnikov.k/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/

In [32]:
query = "Лампа накаливания"
inputs = tokenizer(query, return_tensors="pt").to(device)
with torch.no_grad():
    logits = model(**inputs).logits
    probs = torch.nn.functional.softmax(logits, dim=1).cpu()[0]

In [33]:
torch.topk(probs, k=10)

torch.return_types.topk(
values=tensor([9.8664e-01, 5.7664e-03, 7.7576e-04, 3.7145e-04, 1.6623e-04, 1.0410e-04,
        9.8963e-05, 9.6937e-05, 8.4069e-05, 7.8909e-05]),
indices=tensor([1256, 1254, 1267, 1269, 1097, 1255, 1222,  734, 1297,  673]))

In [34]:
dataset['train'].to_pandas()

,Наименование,Код,Характеристика,Наименование с характеристикой,Ведомственный классификатор,text,label,__index_level_0__
0,Нож,257111.390.000012,специальный,Нож специальный,257111.390,Нож специальный,880,27954
1,Дефектоскоп,265166.930.000002,ультразвуковой,Дефектоскоп ультразвуковой,265166.930,Дефектоскоп ультразвуковой,1146,33164
2,Сопло,282922.300.000003,для пескоструйного аппарата,Сопло для пескоструйного аппарата,282922.300,Сопло для пескоструйного аппарата,1466,9335
3,Термокабель,275129.000.000022,"саморегулирующийся, греющий, удельная мощность...","Термокабель саморегулирующийся, греющий, удель...",275129.000,"Термокабель саморегулирующийся, греющий, удель...",1287,27085
4,Пластина резиновая,221920.700.000050,"тип ОМ, толщина 8 мм","Пластина резиновая тип ОМ, толщина 8 мм",221920.700,"Пластина резиновая тип ОМ, толщина 8 мм",570,32053
...,...,...,...,...,...,...,...,...
19778,Круг,244422.210.000017,"латунный, марка ЛАЖ60-1-1, диаметр 10-180 мм, ...","Круг латунный, марка ЛАЖ60-1-1, диаметр 10-180...",244422.210,"Круг латунный, марка ЛАЖ60-1-1, диаметр 10-180...",820,18485
19779,Переводник для насосно-компрессорных труб,242040.100.000013,"стальной, тип П",Переводник для насосно-компрессорных труб стал...,242040.100,Переводник для насосно-компрессорных труб стал...,775,30741
19780,Известь,235210.330.000007,"негашеная, 2 сорт, комовая, кальциевая, средне...","Известь негашеная, 2 сорт, комовая, кальциевая...",235210.330,"Известь негашеная, 2 сорт, комовая, кальциевая...",701,22332
19781,Накладка рулевой тяги,293230.670.000022,для легкового автомобиля,Накладка рулевой тяги для легкового автомобиля,293230.670,Накладка рулевой тяги для легкового автомобиля,1620,7473
